# LLM for unbiased news summarization - testing notebook

### Setup

In [1]:
# !pip install --upgrade langchain-mistralai langchain langchain_core

In [2]:
import os
import getpass
import json
from IPython.display import Markdown

from langchain_mistralai import ChatMistralAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [3]:
# if "MISTRAL_API_KEY" not in os.environ:
#     os.environ["MISTRAL_API_KEY"] = 'knCyMDokThthiODC2D1vnnUgEsLYVSt1'

### Model and prompts

In [4]:
class LLM:
    '''
    LLM wrapper with robust text generation calls.
    '''
    def __init__(self, model):
        self.model = model

    def generate(self, messages):
        response = None
        while response is None:
            try:
                response = self.model.invoke(messages)
            except Exception as e:
                if 'service_tier_capacity_exceeded' in str(e):
                    continue
                else:
                    print(f'LLM call failed with the following error:\n{str(e)}')
                    return ''

        return response.content

#### SQL query generation

In [5]:
search_system_prompt = SystemMessage(
    content="""Вы являетесь ИИ-экспертом по работе с базами данных. \
Вашей задачей является определение стратегии работы с новостной базой данных под названием "news", которая хранит данные следующего вида:\n\
{
    "id": 12299,
    "date": "2025-11-15T06:34:35+00:00",
    "text": "Текст новости.",
    "raw_text_len": 14,
    "views": 225,
    "forwards": 0,
    "reply_to_msg_id": null,
    "is_reply": false,
    "sender_id": -1001109265956,
    "has_media": true,
    "channel_id": 1109265956,
    "channel_title": "Название канала",
    "channel_username": "channel_username",
    "media": {
      "downloaded": null,
      "media_type": "MessageMediaPhoto"
    }
}\n\
Вам будет дан запрос от пользователя, по которому он хотел бы узнать новости. \
Вам нужно будет определить, какими из следующих методов поиска по базе данных нужно воспользоваться, чтобы извлечь из неё релевантные запросу записи:
а) SQL-поиск: написать SQL-запрос, извлекающий подходящие под запрос записи из базы данных.
б) Поиск по тексту: написать ключевое слово или фразу, по которой можно искать релевантные записи простым поиском по тексту.
в) Векторный поиск: написать текстовый запрос, по которому можно найти похожие записи векторным поиском по их текстам.
По умолчанию сначала выполняется SQL-запрос при его наличии, а затем уже по извлечённым записям выполняется поиск по тексту (либо векторный, либо простой).
В ответ на запрос пользователя Вы выводите JSON следующего вида:
{
  "use_sql": true,
  "sql_query": "SQL-запрос",
  "use_vector_search": true,
  "vector_search_query": "перефразированный запрос для поиска",
  "use_text_search": true,
  "text_search_query": "ключевое слово или фраза"
}
Поля "use_sql", "use_vector_search" и "use_text_search" являются флагами, указывающими на использование соответствующих методов извлечения данных. \
Поле "sql_query" указывается, если "use_sql": true, и содержит SQL-запрос, извлекающий подходящие под запрос записи. \
Если для поиска используется только SQL без поиска по тексту и пользователь явно не задал количество новостей, Ваш SQL-запрос должен извлекать не более 20 записей. \
Если Вы используете SQL с последующим поиском по тексту, то ограничивать количество записей без надобности не нужно.
Поле "vector_search_query" указывается, если "use_vector_search": true, и содержит текст для векторного поиска релевантных запросу пользователя по текстам записей в базе данных. \
Поле "text_search_query" указывается, если "use_text_search": true, и содержит ключевое слово или фразу для простого поиска по тексту. \
Не используйте одновременно простой поиск по тексту и векторный поиск. Если явно задать ключевые слова для простого поиска нельзя, выберите векторный поиск. \
В ответ на запрос пользователя Вы выводите только JSON указанного вида простым текстом без форматирования и комментариев. \
Пожалуйста, не оформляйте итоговый JSON через ```json ```.
Примеры работы:
Пользователь: "Напишите план извлечения из базы данных новостных записей по запросу: 'Новости из канала РБК про экономику'."
Ваш ответ: "{
    "use_sql": true,
    "sql_query": "SELECT * FROM news WHERE channel_title LIKE '%РБК%'",
    "use_vector_search": true,
    "vector_search_query": "экономика",
    "use_text_search": false
}"
Пользователь: "Напишите план извлечения из базы данных новостных записей по запросу: 'Состояние рынка нефти'."
Ваш ответ: "{
    "use_sql": false,
    "use_vector_search": true,
    "vector_search_query": "состояние рынка нефти",
    "use_text_search": false
}"
Пользователь: "Напишите план извлечения из базы данных новостных записей по запросу: 'Новости про Путина'."
Ваш ответ: "{
    "use_sql": false,
    "use_vector_search": false,
    "use_text_search": true,
    "text_search_query": "Путин"
}"
"""
)

In [6]:
def search_prompt(user_query):
    '''
    Return prompt for data search agent with user query.
    '''
    return HumanMessage(
        content=f"Напишите план извлечения из базы данных новостных записей по запросу: '{user_query}'."
    )

In [7]:
def generate_search_plan(user_query, verbose=False):
    '''
    Generate data search plan JSON for given user query.
    '''
    model = ChatMistralAI(
        model="mistral-large-latest",
        max_retries=6,
        api_key='knCyMDokThthiODC2D1vnnUgEsLYVSt1'
    )
    llm = LLM(model)
    
    prompt = search_prompt(user_query)
    messages = [
        search_system_prompt,
        prompt
    ]

    if verbose:
        print('Creating data search plan...')
    plan = llm.generate(messages)
    
    return json.loads(plan)

#### News processing

In [8]:
analyst_system_prompt = SystemMessage(
    content="""Вы являетесь ИИ-экспертом по анализу новостей. \
Ваша работа заключается в анализе новостных источников и представлении информации из них \
в доступной и непредвзятой форме, без каких-либо оценочных суждений, спорных и недоказанных утверждений.\n\
Вам будет представлена подборка новостных сообщений из разных источников по заданной теме. \
Ваша задача будет заключаться в том, чтобы извлечь из них информацию, следуя следующим правилам:\n\
1. Вы предоставляете только подтверждённую информацию из новостей.\n\
2. Если в новостном сообщении присутствуют неподтверждённые заявления, Вы пишете про них в подобной форме: \
"Новостной источник/данный человек заявил, что...".\n\
3. Вы представляете информацию в максимально нейтральной форме.\n\
4. Вы не делаете никаких выводов и оценочных суждений по представленной информации.\n\
5. Вы представляете всю информацию, которую можете получить из предоставленных источников, без цензуры.\n\
6. Источники могут быть на разных языках. Вы представляете всю информацию на русском языке.\n\
7. В ответ на запрос пользователя Вы выводите только информацию из новостей простым текстом \
без форматирования и дополнительных комментариев."""
)

In [9]:
summarizer_system_prompt = SystemMessage(
    content="""Вы являетесь ИИ-агентом, суммаризирующим новостную информацию.\n\
Вам будет представлена представлена информация по определённой теме, собранная из различных \
новостных источников и представленная в сыром виде без оценочных суждений. \
Вашей задачей будет суммаризировать представленную информацию, чтобы представить её в структурированном, \
удобном для чтения пользователем виде. В ответ на запрос пользователя Вы выводите только сводку \
представленной информации без дополнительных комментариев."""
)

In [10]:
def extraction_prompt(user_query, news_posts):
    '''
    Create prompt for inforamtion extraction by analyst agent.
    '''
    prompt = f"Извлеките информацию из следующих новостных сообщений по запросу \"{user_query}\":"
    for i, post in enumerate(news_posts):
        prompt += f"""
Сообщение {i+1}:
- Дата и время: {post['date']}.
- Текст: "{post['text']}".
"""

    return HumanMessage(content=prompt)

In [11]:
def summarization_prompt(user_query, info):
    '''
    Create prompt for summarization agent from given user request and extracted info.
    '''
    return HumanMessage(
        content=f"Составьте сводку следующей новостной информации по запросу \"{user_query}\":\n{info}"
    )

In [12]:
def generate_summary(user_query, news_posts, verbose=False):
    '''
    Generate news summary for given user query with given news posts found by RAG.
    '''
    model = ChatMistralAI(
        model="mistral-large-latest",
        max_retries=6,
        api_key='knCyMDokThthiODC2D1vnnUgEsLYVSt1'
    )
    llm = LLM(model)
    
    prompt_1 = extraction_prompt(user_query, news_posts)
    messages = [
        analyst_system_prompt,
        prompt_1
    ]

    if verbose:
        print('Extracting info from news posts...')
    info = llm.generate(messages)

    prompt_2 = summarization_prompt(user_query, info)
    messages = [
        summarizer_system_prompt,
        prompt_2
    ]

    if verbose:
        print('Summarizing info...')
    summary = llm.generate(messages)
    return summary

### Example

In [13]:
example_query = "Последние новости из канала РБК"
with open('mychannel_messages.json', 'r') as f:
    example_news = json.load(f)[:10]

#### Step-by-step

In [14]:
model = ChatMistralAI(
    model="mistral-large-latest",
    max_retries=6,
    api_key='knCyMDokThthiODC2D1vnnUgEsLYVSt1'
)
llm = LLM(model)

In [15]:
example_extraction_prompt = extraction_prompt(example_query, example_news)
print(example_extraction_prompt.content)

Извлеките информацию из следующих новостных сообщений по запросу "Последние новости из канала РБК":
Сообщение 1:
- Дата и время: 2025-11-14T12:29:47+00:00.
- Текст: "Генконсульство Израиля в Северной столице по просьбе РБК Петербург прокомментировало гипотезу о еврейском происхождении Чебурашки

Резонансную теорию о том, что персонаж Эдуарда Успенского является евреем, на государственный уровень поднял глава думского комитета по бюджету и налогам Андрей Макаров. В качестве аргумента депутат привел коробку с oranges, в которой Чебурашка, согласно сюжету, прибыл в страну.

🟢Генеральный консул Израиля в Петербурге Ран Гидор напомнил, что данные о родителях Чебурашки и его национальности доподлинно неизвестны, однако культовый статус персонажа в Израиле — неоспорим.

«Герой советского мультика стал культовым персонажем и в Израиле, ведь он, метафорически выражаясь, сделал алию, приехав в нашу страну с волной советской эмиграции, — отметил Ран Гидор. — Выходцы из СССР способствовали не толь

In [16]:
messages = [
    analyst_system_prompt,
    example_extraction_prompt
]

example_info = llm.generate(messages)
print(example_info)

Генеральный консул Израиля в Петербурге Ран Гидор прокомментировал гипотезу о еврейском происхождении Чебурашки, поднятую главой думского комитета по бюджету и налогам Андреем Макаровым. Гидор отметил, что данные о родителях и национальности персонажа неизвестны, но Чебурашка стал культовым в Израиле, "метафорически сделав алию" с волной советской эмиграции. Он добавил, что выходцы из СССР способствовали распространению русского языка (сейчас на нём говорит 20% населения Израиля) и культурной экспансии, включая мультипликацию. Гидор заявил, что консульство было бы радо причислить Чебурашку к еврейской нации, но единственный способ это проверить — "удостовериться в наличии обрезания и отсутствии употребления свинины". Ранее подобную теорию выдвигала профессор Майя Балакирски-Кац из Института Туро в Нью-Йорке.

Европейский Союз на фоне коррупционного скандала с украинской госкомпанией "Энергоатом" намерен получить от Украины гарантии по расходованию будущей финансовой помощи, сообщает Po

In [17]:
example_summarization_prompt = summarization_prompt(example_query, example_info)

In [18]:
messages = [
    summarizer_system_prompt,
    example_summarization_prompt
]

example_summary = llm.generate(messages)
Markdown(example_summary)

### **Сводка новостей из канала РБК**

#### **Культура и общество**
- **Чебурашка и еврейское происхождение**: Генеральный консул Израиля в Петербурге Ран Гидор прокомментировал гипотезу о возможном еврейском происхождении Чебурашки, выдвинутую депутатом Андреем Макаровым. Гидор отметил, что персонаж стал культовым в Израиле благодаря волне советской эмиграции, но официальных данных о его национальности нет. Консульство шутливо предложило проверить принадлежность Чебурашки к еврейской нации через "обрезание и отсутствие употребления свинины".

#### **Международные отношения**
- **ЕС требует гарантий от Украины**: На фоне коррупционного скандала с украинской госкомпанией "Энергоатом" Европейский Союз намерен получить от Киева гарантии по расходованию будущей финансовой помощи. Мнения в ЕС разделились: одни видят в разоблачениях знак независимости антикоррупционных органов, другие требуют конкретных обязательств.

- **Египет повышает стоимость виз**: Парламент Египта одобрил законопроект о повышении стоимости виз с $25 до $45 за счёт нового сбора. Туристические организации призвали премьер-министра и министра туризма остановить инициативу.

- **Росатом сообщил об атаке на Нововоронежскую АЭС**: Глава "Росатома" Алексей Лихачёв заявил, что несколько энергоблоков станции были отключены из-за атаки беспилотников в ночь с 12 на 13 ноября. Работа АЭС восстановлена к 11 утра 14 ноября.

- **Россия опровергла обвинения в вербовке иракцев**: Генконсульство России в Эрбиле (Иракский Курдистан) назвало слухами обвинения в вербовке иракцев для участия в боевых действиях на Украине. Представитель консульства заявил, что у миссии нет данных о гибели или участии иракцев в конфликте.

#### **Власть и право**
- **Совет судей рассмотрел отставку высокопоставленных чиновников**: Обсуждалось досрочное прекращение полномочий бывшего председателя Совета судей России Виктора Момотова и ещё 14 членов, а также освобождение Владислава Иванова от должности гендиректора Судебного департамента при Верховном суде. Результаты рассмотрения не озвучены.

- **Оппозиционер Леонид Гозман приговорён к 10 годам колонии**: Суд Москвы заочно приговорил Гозмана (внесён в реестр иноагентов) по делу об оправдании терроризма.

#### **Экономика и потребительский рынок**
- **Шринкфляция обошлась россиянам в 1 трлн рублей**: Союз потребителей России оценил потери от уменьшения объёма товаров при сохранении цен, а также маркетинговых уловок. Организация предложила закрепить требование единообразного оформления всех цен на товар.

#### **Безопасность**
- **Путин обсудил киберугрозы**: Владимир Путин провёл оперативное совещание с постоянными членами Совета Безопасности, где обсуждалась тема противодействия правонарушениям с использованием информационно-коммуникационных технологий.

#### All-in-one

In [19]:
plan = generate_search_plan(example_query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': True, 'sql_query': "SELECT * FROM news WHERE channel_title LIKE '%РБК%' ORDER BY date DESC LIMIT 20", 'use_vector_search': False, 'use_text_search': False}


In [20]:
summary = generate_summary(example_query, example_news, verbose=True)
print("Model ouput:")
Markdown(summary)

Extracting info from news posts...
Summarizing info...
Model ouput:


### **Сводка новостей из канала РБК**

#### **Общество и культура**
- **Генконсул Израиля о Чебурашке**: Ран Гидор прокомментировал гипотезу о еврейском происхождении персонажа, отметив его культовый статус в Израиле и "метафорическую алию" с волной советской эмиграции. Для официального признания Чебурашки евреем потребуется проверка на обрезание и соблюдение кошерных правил.

#### **Международные отношения**
- **ЕС требует от Украины гарантий по антикоррупции**: На фоне скандала в "Энергоатоме" европейские партнёры разделились — одни видят прогресс в борьбе с коррупцией, другие настаивают на конкретных обязательствах.
- **Египет повышает стоимость виз**: Парламент одобрил законопроект о росте цены с $25 до $45, но туристические организации призывают власти остановить инициативу.
- **Россия опровергла вербовку иракцев для войны в Украине**: Генконсульство в Эрбиле назвало обвинения слухами, направленными на ущерб имиджу страны.

#### **Экономика и право**
- **Шринкфляция и маркетинговые уловки**: Союз потребителей оценил потери россиян из-за уменьшения объёма товаров и нечетких ценников в **1 трлн руб.** Предложено ввести единые правила оформления цен.
- **Суд над экс-главой Совета судей**: Рассмотрен вопрос о досрочном прекращении полномочий Виктора Момотова и других чиновников. Ранее у него изъяли недвижимость на **9 млрд руб.** по иску Генпрокуратуры.

#### **Безопасность и правосудие**
- **Атака на Нововоронежскую АЭС**: Беспилотники вывели из строя несколько энергоблоков, но станция восстановлена к 14 ноября.
- **Приговор Леониду Гозману**: Оппозиционера заочно приговорили к **10 годам колонии** по делу об оправдании терроризма.
- **Совет Безопасности РФ**: Владимир Путин обсудил с членами СБ меры против киберпреступлений.

#### More data search agent tests

In [21]:
query = 'Новости США'

plan = generate_search_plan(query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': False, 'use_vector_search': True, 'vector_search_query': 'новости из США или новости о США, включая политическую, экономическую и социальную тематику', 'use_text_search': False}


In [22]:
query = 'Коммерсантъ новости спорта'

plan = generate_search_plan(query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': True, 'sql_query': "SELECT * FROM news WHERE channel_title LIKE '%Коммерсантъ%'", 'use_vector_search': True, 'vector_search_query': 'новости спорта', 'use_text_search': False}


In [23]:
query = 'Новости России за последний день'

plan = generate_search_plan(query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': True, 'sql_query': "SELECT * FROM news WHERE date >= NOW() - INTERVAL '1 day' AND (channel_title LIKE '%Росси%' OR text LIKE '%Росси%') LIMIT 20", 'use_vector_search': False, 'use_text_search': False}


In [24]:
query = 'Новые законопроекты в России'

plan = generate_search_plan(query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': True, 'sql_query': "SELECT * FROM news WHERE date >= NOW() - INTERVAL '6 months' AND (channel_title LIKE '%Дума%' OR channel_title LIKE '%Госдума%' OR channel_title LIKE '%Правительство%' OR channel_title LIKE '%Кремль%' OR channel_title LIKE '%РИА Новости%' OR channel_title LIKE '%ТАСС%' OR channel_title LIKE '%Интерфакс%' OR channel_title LIKE '%РБК%')", 'use_vector_search': True, 'vector_search_query': 'новые законопроекты в России или российские законы, принятые недавно', 'use_text_search': False}


In [25]:
query = 'футбол'

plan = generate_search_plan(query, verbose=True)
print("Model output:\n", plan)

Creating data search plan...
Model output:
 {'use_sql': False, 'use_vector_search': False, 'use_text_search': True, 'text_search_query': 'футбол'}
